# Question 1:  Pose Graph Optimization for 1D SLAM

A solved example for 1D SLAM which optimizes for pose variables using weighted least squares method (Gauss Newton) has been explained in the class. You can view it at `examples/1D-SLAM.ipynb`. Your first task is to code this from scratch.

For this section, you have to calculate Jacobian analytically yourself and use it. However, you can check how correct it is using `jax`'s `jacobian`. Its usage is explained in the supplementary notebook (`examples/`).

## How to get started?

1. Go through the example and understand the optimization procedure.
2. Generate the data as in the example.
3. Write the derivatives of the function with respect to the constraints
4. Create the final jacobian and write the update steps using Gauss Newton

This tiny example will help make sense of the next question, as it's largely just an extension of this.

Finally, plot the loop closure constraint, poses, and error at every iteration as well. Since this is a one dimensional example, you can just assume x = y.

In [2]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import jacfwd

In [ ]:
# Code!
# we will have following steps:
# data generation
# getting info matrix from covariance
# residual 
# jacobian calc
# optimization
# plotting

In [21]:
#data generation
gt_u=[1,1,1,-3,0]
obs_u=[1.1,1,1.1,-2.7,0]
gt_x=[0]
odo_x=[0.0]

def data_generation():
    for _ in range(len(gt_u)-1):
        gt_x.append(gt_u[_]+gt_x[_])
        odo_x.append(obs_u[_]+odo_x[_])

def view_initials():
    # print ("Ground Truth :")
    print("U: ",gt_u)
    print("obs u :" , obs_u)
    print("X: ",gt_x)
    print("Odo x :",odo_x)

data_generation()
view_initials()



U:  [ 1  1  1 -3  0]
obs u : [1.1, 1, 1.1, -2.7, 0]
X:  [0, 1, 2, 3, 0]
Odo x : [0.0, 1.1, 2.1, 3.2, 0.5]


In [36]:
# info matrix from covariance matrix
covariance_matrix=np.array([
    [0.01, 0, 0, 0, 0, 0],
    [0, 0.01, 0, 0, 0, 0],
    [0, 0, 0.01, 0, 0, 0],
    [0, 0, 0, 0.01, 0, 0],
    [0, 0, 0, 0, 0.01, 0],
    [0, 0, 0, 0, 0, 0.001]
])
# weights = jnp.diag(jnp.array([.01, .01, .01,.01, .01, .001]))
# print(weights)
residual=[]

info_matrix=np.linalg.inv(covariance_matrix)


def calc_residual(u,x):
    residual=[]
    for i in range(len(u)-1):
        residual.append(u[i]+x[i]-x[i+1])
    residual.append(u[4]+x[0]-x[4])
    residual.append(x[0]-0)
    res=jnp.array(residual)
    return res
    
print("Info matrix : \n",info_matrix)



Info matrix : 
 [[ 100.    0.    0.    0.    0.    0.]
 [   0.  100.    0.    0.    0.    0.]
 [   0.    0.  100.    0.    0.    0.]
 [   0.    0.    0.  100.    0.    0.]
 [   0.    0.    0.    0.  100.    0.]
 [   0.    0.    0.    0.    0. 1000.]]


In [33]:
#jacobian

analytical_j=np.array(
    [
        [1, -1, 0, 0, 0],
        [0, 1, -1, 0, 0],
        [0, 0, 1, -1, 0],
        [0, 0, 0, 1, -1],
        [1, 0, 0, 0, -1],
        [1, 0, 0, 0, 0]
    ]
)

# Jacobian using Jax
def jax_J():
    global obs_u
    global odo_x
    resd=lambda X: calc_residual(obs_u, X)
    func_J = jacfwd(resd)
    jax_from_funct=np.array(func_J(odo_x)).T
    return jax_from_funct




print("analytical_jacobian is :\n",analytical_j)

print("jax j is :\n",jax_J())




analytical_jacobian is :
 [[ 1 -1  0  0  0]
 [ 0  1 -1  0  0]
 [ 0  0  1 -1  0]
 [ 0  0  0  1 -1]
 [ 1  0  0  0 -1]
 [ 1  0  0  0  0]]
jax j is :
 [[ 1. -1.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.]
 [ 0.  0.  1. -1.  0.]
 [ 0.  0.  0.  1. -1.]
 [ 1.  0.  0.  0. -1.]
 [ 1.  0.  0.  0.  0.]]


In [52]:
def gauss_newton(info_matrix,U,X,iter=10):
    x_list=[X[0]]
    print("Intial X: ",X[0])
    H=analytical_j.T @ info_matrix @ analytical_j
    for _ in range(iter):
        b= analytical_j.T @ info_matrix.T @ calc_residual(U[0],X[0])
        b=b.reshape((-1,1))
        del_x= -1* jnp.linalg.inv(H) @ b
        X= X+ del_x.reshape((-1,1))
        x_list.append(X[0])

    with jnp.printoptions(precision=7, suppress=True):
        print("Final X = \n",X[0])

    return jnp.array(x_list)

analytical=gauss_newton(info_matrix,jnp.array(obs_u).reshape((1,-1)),jnp.array(odo_x).reshape((1,-1)))
print(analytical)


Intial X:  [0.  1.1 2.1 3.2 0.5]
Final X = 
 [-0.   1.1  2.1  3.2  0.5]
[[ 0.0000000e+00  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-1.6298145e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-3.2596290e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-1.1641532e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-2.7939677e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-6.9849193e-10  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-2.3283064e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-2.3283064e-10  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-1.8626451e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-3.4924597e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]
 [-1.3969839e-09  1.1000000e+00  2.0999999e+00  3.2000000e+00
   5.0000000e-01]]
